In [1]:
import numpy as np
from data import load_dataset, get_train_test_split

In [2]:
filename = "data/LoanStats3a.csv"
data = load_dataset(filename)
print("Data shape: %s" % str(data.shape))

Preprocessing...
Feature Engineering...
Data shape: (42535, 39)


In [3]:
X_train, X_test, y_train, y_test = get_train_test_split(data)
print("X_train: %s, y_train: %s" % (str(X_train.shape), str(y_train.shape)))
print("X_test: %s, y_test: %s" % (str(X_test.shape), str(y_test.shape)))

X_train: (29774, 37), y_train: (29774,)
X_test: (12761, 37), y_test: (12761,)


In [6]:
# TODO: Implement regression model
from sklearn import svm
# Initialize SVM
model = svm.SVR()

# Train the model
model.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [7]:
pred = model.predict(X_train)
mean_abs_err = np.mean(np.abs(pred - y_train))
print("Mean absolute error: %.2f" % mean_abs_err)

Mean absolute error: 6741.87
